## Pandas缺失值处理

导包

In [1]:
import numpy as np
import pandas as pd

有两种丢失数据(空值)：
- None
- np.nan

### 1. None
- None是Python自带的，是Python中的空对象。None不能参与到任何计算中。
- object类型的运算要比int类型的运算慢得多  

In [2]:
%timeit np.arange(1e6, dtype=object).sum()

52.8 ms ± 851 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
%timeit np.arange(1e6, dtype=np.int32).sum()

1.54 ms ± 28.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### 2. np.nan

- np.nan是浮点类型，能参与到计算中。但计算的结果总是NaN。

In [4]:
type(np.nan)

float

- 但可以使用np.nan*()函数来计算nan，此时会过滤掉nan。

In [5]:
n = np.array([1, 2, 3, np.nan, 5, 6])
n

array([ 1.,  2.,  3., nan,  5.,  6.])

In [6]:
np.sum(n)  # nan
np.nansum(n)  # 自动过滤nan不计算

17.0

In [7]:
np.nan + 10

nan

### 3. Pandas中的None与NaN

#### 1) Pandas中None与np.nan都视作np.nan

- 创建DataFrame

In [8]:
data = np.random.randint(0, 100, size=(5, 5))
df = pd.DataFrame(data=data, columns=list('ABCDE'))
df

,A,B,C,D,E
0,29,1,21,51,45
1,98,19,59,40,64
2,74,71,29,35,40
3,22,49,4,86,55
4,42,28,36,38,44


- 使用DataFrame行索引与列索引修改DataFrame数据

In [9]:
df.loc[2, 'B'] = np.nan
df.loc[3, 'C'] = None
df

,A,B,C,D,E
0,29,1.0,21.0,51,45
1,98,19.0,59.0,40,64
2,74,NaN,29.0,35,40
3,22,49.0,NaN,86,55
4,42,28.0,36.0,38,44


In [10]:
df.loc[2, 'B'], df.loc[3, 'C']

(nan, nan)

#### 2) pandas中None与np.nan的操作

- isnull()
- notnull()
- all()
- any()
- dropna():  过滤丢失数据
- fillna():  填充丢失数据


(1)判断函数

- isnull()
- notnull()


In [11]:
df

,A,B,C,D,E
0,29,1.0,21.0,51,45
1,98,19.0,59.0,40,64
2,74,NaN,29.0,35,40
3,22,49.0,NaN,86,55
4,42,28.0,36.0,38,44


In [12]:
df.isnull()

,A,B,C,D,E
0,False,False,False,False,False
1,False,False,False,False,False
2,False,True,False,False,False
3,False,False,True,False,False
4,False,False,False,False,False


In [13]:
df.notnull()

,A,B,C,D,E
0,True,True,True,True,True
1,True,True,True,True,True
2,True,False,True,True,True
3,True,True,False,True,True
4,True,True,True,True,True


In [14]:
# all()  : 必须全部为True才会是True，类似and
# any() : 只要有一个为True就为True，类似or

# 找有空的列
df.isnull().any()   # 常用，尽可能找到有空的列或行
# df.isnull().all()  # 必须全部都为空的行或列才会为True

# 找没有空的列
df.notnull().all()  # 常用，尽量找没有空值的列或行
# df.notnull().any()

A     True
B    False
C    False
D     True
E     True
dtype: bool

In [15]:
# 找有空的行
df.isnull().any(axis=1)

# 找没有空的行
df.notnull().all(axis=1)

0     True
1     True
2    False
3    False
4     True
dtype: bool

- 使用bool值索引过滤数据

In [16]:
# 过滤数据
df

,A,B,C,D,E
0,29,1.0,21.0,51,45
1,98,19.0,59.0,40,64
2,74,NaN,29.0,35,40
3,22,49.0,NaN,86,55
4,42,28.0,36.0,38,44


In [17]:
# 行过滤
cond = df.isnull().any(axis=1)
# display(~cond)  # 取反
df[~cond]

cond = df.notnull().all(axis=1)
df[cond]

,A,B,C,D,E
0,29,1.0,21.0,51,45
1,98,19.0,59.0,40,64
4,42,28.0,36.0,38,44


In [18]:
df

# 过滤列
cond = df.isnull().any()
df.loc[:, ~cond]

cond = df.notnull().all()
df.loc[:, cond]

,A,D,E
0,29,51,45
1,98,40,64
2,74,35,40
3,22,86,55
4,42,38,44


(2) 过滤函数
- dropna()

可以选择过滤的是行还是列（默认为行）

In [19]:
df

,A,B,C,D,E
0,29,1.0,21.0,51,45
1,98,19.0,59.0,40,64
2,74,NaN,29.0,35,40
3,22,49.0,NaN,86,55
4,42,28.0,36.0,38,44


In [20]:
# 默认删除有空的行
df.dropna()

df.dropna(axis=1)  # 删除有空的列

,A,D,E
0,29,51,45
1,98,40,64
2,74,35,40
3,22,86,55
4,42,38,44


也可以选择过滤的方式 how = 'all'

In [21]:
df.dropna(how='any')

# 必须所有数据都为nan才会删除
df.dropna(how='all', axis=1)

,A,B,C,D,E
0,29,1.0,21.0,51,45
1,98,19.0,59.0,40,64
2,74,NaN,29.0,35,40
3,22,49.0,NaN,86,55
4,42,28.0,36.0,38,44


inplace=True 修改原数据

In [22]:
df2 = df.copy()
df2

# inplace=True 修改原数据
df2.dropna(inplace=True)
df2

,A,B,C,D,E
0,29,1.0,21.0,51,45
1,98,19.0,59.0,40,64
4,42,28.0,36.0,38,44


(3) 填充函数 Series/DataFrame
- fillna()

In [23]:
df

,A,B,C,D,E
0,29,1.0,21.0,51,45
1,98,19.0,59.0,40,64
2,74,NaN,29.0,35,40
3,22,49.0,NaN,86,55
4,42,28.0,36.0,38,44


In [24]:
# 填充nan
df.fillna(value=100)

,A,B,C,D,E
0,29,1.0,21.0,51,45
1,98,19.0,59.0,40,64
2,74,100.0,29.0,35,40
3,22,49.0,100.0,86,55
4,42,28.0,36.0,38,44


In [25]:
df

,A,B,C,D,E
0,29,1.0,21.0,51,45
1,98,19.0,59.0,40,64
2,74,NaN,29.0,35,40
3,22,49.0,NaN,86,55
4,42,28.0,36.0,38,44


In [26]:
df2 = df.copy()
df2.loc[1, 'B'] = np.nan
df2.loc[2, 'C'] = np.nan
df2

,A,B,C,D,E
0,29,1.0,21.0,51,45
1,98,NaN,59.0,40,64
2,74,NaN,NaN,35,40
3,22,49.0,NaN,86,55
4,42,28.0,36.0,38,44


In [27]:
# limit: 限制填充的次数
df2.fillna(value=100, limit=1, inplace=True)

In [28]:
df2

,A,B,C,D,E
0,29,1.0,21.0,51,45
1,98,100.0,59.0,40,64
2,74,NaN,100.0,35,40
3,22,49.0,NaN,86,55
4,42,28.0,36.0,38,44


可以选择前向填充还是后向填充

In [29]:
df

,A,B,C,D,E
0,29,1.0,21.0,51,45
1,98,19.0,59.0,40,64
2,74,NaN,29.0,35,40
3,22,49.0,NaN,86,55
4,42,28.0,36.0,38,44


In [30]:
df.fillna(method='ffill')  # 向前填充
df.fillna(method='backfill')  # 向后填充

# method : {'backfill', 'bfill', 'pad', 'ffill', None}, default None
#     Method to use for filling holes in reindexed Series
#     pad / ffill: propagate last valid observation forward to next valid
#     backfill / bfill: use next valid observation to fill gap.

,A,B,C,D,E
0,29,1.0,21.0,51,45
1,98,19.0,59.0,40,64
2,74,49.0,29.0,35,40
3,22,49.0,36.0,86,55
4,42,28.0,36.0,38,44


In [31]:
df.fillna(method='ffill', axis=1)  # 向左填充
df.fillna(method='backfill', axis=1)  # 向右填充

,A,B,C,D,E
0,29.0,1.0,21.0,51.0,45.0
1,98.0,19.0,59.0,40.0,64.0
2,74.0,29.0,29.0,35.0,40.0
3,22.0,49.0,86.0,86.0,55.0
4,42.0,28.0,36.0,38.0,44.0
